## A. Identification of Candidate Datasets

1. Wordbank-An open database of children's vocabulary development
- Course: Text mining, embeddings
- Beyond course: Natural Language
- Dataset size and structure: There are over 7 million entries. The row include information about the child as well as the word that they are trying to learn.
- Data types: Mainly text. There are a few attributes such as child age that are numeric.
- Target variables: Child attributes when a word is learned.
- Licensing and usage constraints: CC BY 4.0. If used in a derivative work, credit must be given. Otherwise, the dataset is free to use.
2. YouTube Video Social Graph
- Course: Graph Mining, PageRank, Community Detection
- Beyond course: Link Prediction
- Dataset size and structure: About 60 zip files of different crawls. Each file typically has 60,000-100,000 videos. Each entry has information about the current video and 20 videoIds that show up in the current video's recommened videos.
- Data types: Mainly text. There are a few attributes such as views that are numeric.
- Target variables: The videoIds in the a given video's recommended videos section.
- Licensing and usage constraints: None specified.
3. Amazon Sales Dataset
- Course: Frequent Itemsets, Association Rules, Text mining
- Beyond course: Sentiment-Augmented Rules
- Dataset size and structure: There are about 1500 entries. Each entry gives product information, the buyer, and the review that the buyer wrote.
- Data types: A split between numeric and text.
- Target variables: Product rating given a userId and a productId.
- Licensing and usage constraints: CC BY-NC-SA 4.0. Users must credit the creator and indicate any changes when redistibuting the dataset.

## Resources.

On my honor, I declare the following resources:
1. Collaborators:
None

2. Web Sources:
- https://wordbank.stanford.edu/data/
- https://netsg.cs.sfu.ca/youtubedata/
- https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset/data

3. AI Tools:
None

## B. Comparative Analysis of Datasets

1. Wordbank-An open database of children's vocabulary development
- Supported data mining tasks: Text mining (course), embeddings (course), natural language processing (external)
- Data quality issues: Some words are sounds such as "baa baa" which could lead to variations considered different words.
- Algorithmic feasibility: 
- Bias considerations:
- Ethical considerations:

2. Youtube Video Social Graph
- Supported data mining tasks:
- Data quality issues:
- Algorithmic feasibility:
- Bias considerations:
- Ethical considerations:

3. Amazon Sales Dataset
- Supported data mining tasks:
- Data quality issues:
- Algorithmic feasibility:
- Bias considerations:
- Ethical considerations:

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.95M/1.95M [00:00<00:00, 4.66MB/s]

Extracting files...


Path to dataset files: C:\Users\zacha\.cache\kagglehub\datasets\karkavelrajaj\amazon-sales-dataset\versions\1
